In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [2]:
# Data Augmentation for Training
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
training_set = train_datagen.flow_from_directory(
    "Monkeypox/archive (60)/Augmented Images/Augmented Images/FOLDS_AUG/fold4_AUG/Train/",
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',  # Use 'categorical' for one-hot encoded labels
)

Found 7336 images belonging to 6 classes.


In [3]:
test_datagen = ImageDataGenerator(rescale=1./255)
validation_set = test_datagen.flow_from_directory(
    "Monkeypox/archive (60)/Augmented Images/Augmented Images/FOLDS_AUG/fold4_AUG/Train/",
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',  # Use 'categorical' for one-hot encoded labels
)

Found 7336 images belonging to 6 classes.


In [4]:
# Custom F1 Score Metric
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        # Update the precision and recall for each batch
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        # Calculate F1 score as the harmonic mean of precision and recall
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()


In [5]:
# Build the GRU Model
gru_model = tf.keras.models.Sequential()

In [6]:
# Flatten and Reshape for GRU
gru_model.add(tf.keras.layers.Flatten(input_shape=(64, 64, 3)))
gru_model.add(tf.keras.layers.Reshape((16, -1)))  # Reshape to (time_steps, features)

C:\Users\KIIT\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [7]:
# GRU Layer
gru_model.add(tf.keras.layers.GRU(units=64, activation='tanh', return_sequences=False))

In [8]:
# Fully Connected Layers
gru_model.add(tf.keras.layers.Dense(units=128, activation='relu'))
gru_model.add(tf.keras.layers.Dense(6, activation='softmax'))  # 6 classes

In [9]:
# Compile the model with Precision, Recall, and F1 Score
gru_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',  # Use categorical_crossentropy for one-hot encoded labels
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score(name='f1_score')  # Add custom F1 score metric
    ]
)

In [10]:
# Train the model
history = gru_model.fit(
    x=training_set,
    validation_data=validation_set,
    epochs=100
)

C:\Users\KIIT\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
230/230 ━━━━━━━━━━━━━━━━━━━━ 0s 282ms/step - accuracy: 0.3479 - f1_score: 0.0476 - loss: 1.6242 - precision: 0.4340 - recall: 0.0253

C:\Users\KIIT\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


230/230 ━━━━━━━━━━━━━━━━━━━━ 109s 426ms/step - accuracy: 0.3480 - f1_score: 0.0477 - loss: 1.6240 - precision: 0.4343 - recall: 0.0253 - val_accuracy: 0.3886 - val_f1_score: 0.1278 - val_loss: 1.5412 - val_precision: 0.5544 - val_recall: 0.0722
Epoch 2/100
230/230 ━━━━━━━━━━━━━━━━━━━━ 105s 457ms/step - accuracy: 0.3886 - f1_score: 0.1240 - loss: 1.5446 - precision: 0.5438 - recall: 0.0701 - val_accuracy: 0.3939 - val_f1_score: 0.0951 - val_loss: 1.5141 - val_precision: 0.6856 - val_recall: 0.0511
Epoch 3/100
230/230 ━━━━━━━━━━━━━━━━━━━━ 87s 379ms/step - accuracy: 0.4179 - f1_score: 0.1202 - loss: 1.5141 - precision: 0.5532 - recall: 0.0679 - val_accuracy: 0.4463 - val_f1_score: 0.3026 - val_loss: 1.4319 - val_precision: 0.5589 - val_recall: 0.2075
Epoch 4/100
230/230 ━━━━━━━━━━━━━━━━━━━━ 84s 366ms/step - accuracy: 0.4568 - f1_score: 0.2602 - loss: 1.4433 - precision: 0.5933 - recall: 0.1670 - val_accuracy: 0.4757 - val_f1_score: 0.2372 - val_loss: 1.4019 - val_precision: 0.6374 - val_r

In [13]:
# Save Loss plot
plt.figure()
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title("Loss")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.savefig('loss_plot_gru.png')  # Save the plot as a PNG file
plt.close()

# Save Accuracy plot
plt.figure()
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title("Accuracy")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.savefig('accuracy_plot_gru.png')  # Save the plot as a PNG file
plt.close()

# Save Precision plot
plt.figure()
plt.plot(history.history['precision'], label='Train Precision')
plt.plot(history.history['val_precision'], label='Validation Precision')
plt.legend()
plt.title("Precision")
plt.xlabel('Epochs')
plt.ylabel('Precision')
plt.savefig('precision_plot_gru.png')  # Save the plot as a PNG file
plt.close()

# Save Recall plot
plt.figure()
plt.plot(history.history['recall'], label='Train Recall')
plt.plot(history.history['val_recall'], label='Validation Recall')
plt.legend()
plt.title("Recall")
plt.xlabel('Epochs')
plt.ylabel('Recall')
plt.savefig('recall_plot_gru.png')  # Save the plot as a PNG file
plt.close()

# Save F1 Score plot
plt.figure()
plt.plot(history.history['f1_score'], label='Train F1 Score')
plt.plot(history.history['val_f1_score'], label='Validation F1 Score')
plt.legend()
plt.title("F1 Score")
plt.xlabel('Epochs')
plt.ylabel('F1 Score')
plt.savefig('f1_score_plot_gru.png')  # Save the plot as a PNG file
plt.close()


In [14]:
gru_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten (Flatten)                    │ (None, 12288)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 16, 768)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ (None, 64)                  │         160,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 6)                   │             774 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 507,668 (1.94 MB)

 Trainable params: 169,222 (661.02 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 338,446 (1.29 MB)